# Analysis od the hypercube

In [ ]:
#import functions
import sys
import glob
import numpy as np
import os


#import local functions
sys.path.append('../../fonction/') 
from data_visualisation import *
from load_data import *
from matrix_tools import *
from walsh_hadamard import *
from walsh_ordered_recon import *

import pysptools as pysptools
from pysptools import noise,abundance_maps

In [ ]:
%matplotlib 
#inline 

## Load data

In [ ]:
%%time
Nl = 512 # number of pixcels along the y dimensions 
Nc = 512 # number of measured Walsh_Hadmard coefficients (correpond to the h dimensions)
Nd = 128 #

load_path = '../../data/2023_02_28_mRFP_DsRed_3D/Reconstruction/hyper_cube/'
list_run = ['RUN0004','RUN0005','RUN0006','RUN0007','RUN0008','RUN0009','RUN0010','RUN0011','RUN0012','RUN0013','RUN0014','RUN0015','RUN0016','RUN0017','RUN0018','RUN0019','RUN0020','RUN0021','RUN0022','RUN0023','RUN0024']
N_run = len(list_run)
ivD_cube = np.zeros((Nl*Nc*N_run,Nd))
i = 0
for run in list_run :
    #a = 1
    file_name = run+"_rc_pinv_"+str(Nl)+"x"+str(Nc)+"x"+str(Nd)+".npy"
    rc_hyper_cube = np.load(load_path+file_name)
    non_neg = np.where(rc_hyper_cube > 0, rc_hyper_cube, 0)
    #data_size = Nl*Nc
    
    for l in range(Nl):
        for c in range(Nc):
            ivD_cube[i] = non_neg[l,c]
            i += 1

## Load reference spectrum

In [ ]:
EGFP_path = "../../data/Reference_spectra//EGFP_fpbase_spectra_500_608.csv"
DsRed2_path = "../../data/Reference_spectra/DsRed2_fpbase_spectra_500_608.csv"
DsRed_path = "../../data/Reference_spectra//DsRed_fpbase_spectra_500_608.csv"
DsRed_exp_path = "../../data/Reference_spectra//DsRed_express_fpbase_spectra_550_658.csv" 
mCherry_path = "../../data/Reference_spectra//mCherry_fpbase_spectra_550_658.csv"
mRFP_path =  "../../data/Reference_spectra//mRFP1_fpbase_spectra_550_658.csv"

data_DsRed_exp = np.genfromtxt(DsRed_exp_path, delimiter=';',skip_header=True)#,
data_DsRed = np.genfromtxt(DsRed_path, delimiter=',',skip_header=True)
data_DsRed2 = np.genfromtxt(DsRed2_path, delimiter=',',skip_header=True)
data_mCherry = np.genfromtxt(mCherry_path, delimiter=';',skip_header=True)
data_EGFP = np.genfromtxt(EGFP_path, delimiter=',',skip_header=True)
data_mRFP = np.genfromtxt(mRFP_path, delimiter=';',skip_header=True)

In [ ]:
L_lambda = [550+i*(108/(Nd-1)) for i in range(Nd)]

In [ ]:
L_dim = np.zeros(Nd)
L = data_mRFP[:,1]#data_mCherry[:,1]
for i in range(Nd):
    val = L_lambda[i]
    int_val = int(val)
    if val-int_val == 0.0:
        L_dim[i] = L[int_val-550]
    else:
        poid = val%1
        L_dim[i] = (1-poid)*L[int_val-550]+poid*L[int_val-550+1]
simple_plot_2D(L_lambda,L_dim)
L_mRFP = L_dim

In [ ]:
L_dim = np.zeros(Nd)
L = data_DsRed_exp[:,1]#
for i in range(Nd):
    val = L_lambda[i]
    int_val = int(val)
    if val-int_val == 0.0:
        L_dim[i] = L[int_val-550]
    else:
        poid = val%1
        L_dim[i] = (1-poid)*L[int_val-550]+poid*L[int_val-550+1]
        
L_DsRed_exp = L_dim
simple_plot_2D(L_lambda,L_dim)

## Estimate auto-fluoresence

In [ ]:
Save_path = '../../data/2023_02_28_mRFP_DsRed_3D/Reconstruction/hyper_cube/'
list_run = ['RUN0004','RUN0005','RUN0006']

spec_fluo = np.zeros(128)
for run in list_run:
    file_name = run+"_rc_pinv_"+str(Nl)+"x"+str(Nc)+"x"+str(Nd)+".npy"
    rc_hyper_cube = np.load(Save_path+file_name)
    for l in range(235,285,1):
        for c in range(320,345,1):
            spec_fluo += rc_hyper_cube[l,c]

simple_plot_2D(L_lambda,spec_fluo)

## Spectral unmixing

### Endmember matrix

In [ ]:
comp = 4
H = np.zeros((comp,Nd))
H[0] = L_DsRed_exp
H[1] = L_mRFP
H[2] = spec_fluo/np.max(spec_fluo)
H[3] = np.ones(128)*1/10

U = H
M = ivD_cube

In [ ]:
plt.figure(1)
L_comp = ['DsRed2','mRFP','AF','Noise']
plt.clf()
plt.plot(L_lambda,H[0],'r',label = L_comp[0])
plt.plot(L_lambda, H[1], 'g', label = L_comp[1])
plt.plot(L_lambda, H[2],'b', label = L_comp[2])
plt.plot(L_lambda, H[3],'c', label = L_comp[3])

plt.legend()
plt.xlabel('Wavelenght (nm)')
plt.ylabel('Intensitiy (normalized)')

plt.show()

### Unmixing

In [ ]:
%%time
Abondance = abundance_maps.amaps.NNLS(M, U)

In [ ]:
resto_4D = np.zeros((Nl,Nc,N_run,comp))
for ncomp in range(comp):
    i = 0 
    for run in range(N_run):
        for l in range(Nl):
            for c in range(Nc):
                resto_4D[l,c,run,ncomp] =Abondance[i,ncomp] 
                i += 1

In [ ]:
lisrt_fluo = ['Dsred','mRFP','AF','Noise']
j = 0
for Fluo in lisrt_fluo: 
    
    save_path = '../../data/2023_02_28_mRFP_DsRed_3D/Analyse_out/spec_unmix/'
    save_path_im = '../../data/2023_02_28_mRFP_DsRed_3D/Analyse_out/spec_unmix/'+str(Fluo)+'/'
    list_run = ['RUN0004','RUN0005','RUN0006','RUN0007','RUN0008','RUN0009','RUN0010','RUN0011','RUN0012','RUN0013','RUN0014','RUN0015','RUN0016','RUN0017','RUN0018','RUN0019','RUN0020','RUN0021','RUN0022','RUN0023','RUN0024']
    
    if not os.path.exists(save_path_im):
        os.makedirs(save_path_im)
        
    np.save(save_path+'Spectre_endmember_'+str(Fluo)+'.npy',H[j])
    for i in range(N_run):
 
        np.save(save_path_im+list_run[i]+"_abond_map_"+Fluo+".npy",resto_4D[:,:,i,j],)
        #im_save = resto_4D[:,:,i,j]
        #hdu = fits.PrimaryHDU(im_save)
        #hdul = fits.HDUList([hdu])
        #hdul.writeto(save_path_im+"RUN_"+str(i+4)+"_abond_map_"+Fluo+".fits")
    j+=1

## Filter unmixing

In [ ]:
# mRFP DsRed filters
Filt = np.zeros((128,2))
N_filt = 2
Filt[40:52,0] = np.ones(abs(52-40))
Filt[62:74,1] = np.ones(abs(62-74))

In [ ]:
Filt_hyper = np.dot(ivD_cube,Filt)

In [ ]:
resto_4D = np.zeros((Nl,Nc,N_run,comp))
for ncomp in range(N_filt):
    i = 0 
    for run in range(N_run):
        for l in range(Nl):
            for c in range(Nc):
                resto_4D[l,c,run,ncomp] = Filt_hyper[i,ncomp] 
                i += 1

In [ ]:
for i in range(N_run):
    plot_im2D(resto_4D[:,:,i,1],cmap='gray',title='run '+str(i))

In [ ]:
lisrt_fluo = ['Dsred','mRFP']
j = 0
for Fluo in lisrt_fluo: 
    
    save_path = '../../data/2023_02_28_mRFP_DsRed_3D/Analyse_out/Filt_sep/'
    save_path_im = '../../data/2023_02_28_mRFP_DsRed_3D/Analyse_out/Filt_sep/'+str(Fluo)+'/'
    
    if not os.path.exists(save_path_im):
        os.makedirs(save_path_im)
        
    np.save(save_path+'Filter_'+str(Fluo)+'.npy',Filt[j])
    for i in range(N_run):
        #im_save = resto_4D[:,:,i,j]
        #hdu = fits.PrimaryHDU(im_save)
        #hdul = fits.HDUList([hdu])
        #hdul.writeto(save_path_im+"RUN"+str(i+4)+"_Filt_"+Fluo+".fits")
        np.save(save_path_im+"RUN"+str(i+4)+"_Filt_"+Fluo+".npy",resto_4D[:,:,i,j],)
        
    j+=1